In [1]:
import pandas as  pd
import seaborn as sns

from category_encoders.target_encoder import TargetEncoder
from category_encoders.binary import BinaryEncoder
from category_encoders.one_hot import OneHotEncoder

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [2]:
 import warnings; warnings.simplefilter('ignore')

In [3]:
data = pd.read_csv('train.csv')
data

,age,job,marital,education,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,57,management,married,tertiary,320,no,no,cellular,7,aug,76,2,-1,0,unknown,no
1,38,management,divorced,tertiary,10638,no,no,unknown,12,jun,159,2,-1,0,unknown,no
2,44,services,divorced,secondary,608,yes,yes,unknown,18,jun,19,30,-1,0,unknown,no
3,44,technician,divorced,unknown,1823,no,no,unknown,20,jun,43,2,-1,0,unknown,no
4,38,admin.,divorced,secondary,1314,no,no,cellular,29,aug,156,2,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27121,56,technician,married,tertiary,272,no,no,cellular,26,aug,103,2,-1,0,unknown,no
27122,36,blue-collar,married,primary,250,yes,no,unknown,8,may,149,1,-1,0,unknown,no
27123,34,technician,single,secondary,1161,yes,no,cellular,20,apr,17,5,334,1,failure,no
27124,29,management,married,tertiary,198,yes,no,cellular,3,aug,1179,1,-1,0,unknown,no


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27126 entries, 0 to 27125
Data columns (total 16 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        27126 non-null  int64 
 1   job        27126 non-null  object
 2   marital    27126 non-null  object
 3   education  27126 non-null  object
 4   balance    27126 non-null  int64 
 5   housing    27126 non-null  object
 6   loan       27126 non-null  object
 7   contact    27126 non-null  object
 8   day        27126 non-null  int64 
 9   month      27126 non-null  object
 10  duration   27126 non-null  int64 
 11  campaign   27126 non-null  int64 
 12  pdays      27126 non-null  int64 
 13  previous   27126 non-null  int64 
 14  poutcome   27126 non-null  object
 15  y          27126 non-null  object
dtypes: int64(7), object(9)
memory usage: 3.3+ MB


In [5]:
data.y = data.y.replace('no', 0)
data.y = data.y.replace('yes', 1)

In [6]:
housing_encoder = BinaryEncoder()
housing_encoder.fit(data['housing'])
housing_encoded = housing_encoder.transform(data['housing'])
for i in housing_encoded:
    data[i] = housing_encoded[i].astype('category')
del data['housing']

In [7]:
loan_encoder = BinaryEncoder()
loan_encoder.fit(data['loan'])
loan_encoded = loan_encoder.transform(data['loan'])
for i in loan_encoded:
    data[i] = loan_encoded[i].astype('category')
del data['loan']

In [8]:
marital_encoder = TargetEncoder()
marital_encoder.fit(data['marital'], data['y'])
data['marital'] = marital_encoder.transform(data['marital']).astype('category')

In [9]:
education_encoder = TargetEncoder()
education_encoder.fit(data.education, data.y)
data.education = education_encoder.transform(data.education).astype('category')

In [10]:
job_encoder = TargetEncoder()
job_encoder.fit(data.job, data.y)
data.job = job_encoder.transform(data.job).astype('category')

In [11]:
contact_encoder = TargetEncoder()
contact_encoder.fit(data.contact, data.y)
data.contact = contact_encoder.transform(data.contact).astype('category')

In [12]:
poutcome_encoder = TargetEncoder()
poutcome_encoder.fit(data.poutcome, data.y)
data.poutcome = poutcome_encoder.transform(data.poutcome).astype('category')

In [13]:
month_encoder = TargetEncoder()
month_encoder.fit(data.month, data.y)
data.month = month_encoder.transform(data.month).astype('category')

In [14]:
day_encoder = TargetEncoder()
day_encoder.fit(data.day, data.y)
data.day = day_encoder.transform(data.day).astype('category')

In [15]:
data.corr()['y']

age          0.028202
job          0.136888
marital      0.059412
education    0.066694
balance      0.039490
contact      0.153403
day         -0.028128
month        0.264083
duration     0.387692
campaign    -0.069738
pdays        0.099874
previous     0.117769
poutcome     0.321685
y            1.000000
housing_0   -0.141707
housing_1    0.141707
loan_0      -0.067327
loan_1       0.067327
Name: y, dtype: float64

In [16]:
data.columns

Index(['age', 'job', 'marital', 'education', 'balance', 'contact', 'day',
       'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y',
       'housing_0', 'housing_1', 'loan_0', 'loan_1'],
      dtype='object')

In [17]:
X_train = data[['age', 'job', 'marital', 'education', 'balance', 'contact', 'day',
       'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome',
       'housing_0', 'housing_1', 'loan_0', 'loan_1']]
y_train = data['y']

In [18]:
standart = StandardScaler()
standart.fit(X_train)
standarted = standart.transform(X_train)

X_train = pd.DataFrame(data=standarted)

In [19]:
X_train.columns = ['age', 'job', 'marital', 'education', 'balance', 'contact', 'day',
       'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome',
       'housing_0', 'housing_1', 'loan_0', 'loan_1']
X_train

,age,job,marital,education,balance,contact,day,month,duration,campaign,pdays,previous,poutcome,housing_0,housing_1,loan_0,loan_1
0,1.513639,0.388501,-0.713079,1.477503,-0.331085,0.662612,-1.054741,-0.112855,-0.714964,-0.246027,-0.409337,-0.294787,-0.245147,-1.123727,1.123727,-0.437623,0.437623
1,-0.276014,0.388501,-0.074418,1.477503,2.977626,-1.552032,-0.453116,-0.226236,-0.388568,-0.246027,-0.409337,-0.294787,-0.245147,-1.123727,1.123727,-0.437623,0.437623
2,0.289140,-0.694305,-0.074418,-0.475788,-0.238731,-1.552032,0.268835,-0.226236,-0.939116,8.814487,-0.409337,-0.294787,-0.245147,0.889896,-0.889896,2.285073,-2.285073
3,0.289140,-0.142211,-0.074418,0.238931,0.150888,-1.552032,0.509486,-0.226236,-0.844736,-0.246027,-0.409337,-0.294787,-0.245147,-1.123727,1.123727,-0.437623,0.437623
4,-0.276014,0.202979,-0.074418,-0.475788,-0.012335,0.662612,1.592412,-0.112855,-0.400365,-0.246027,-0.409337,-0.294787,-0.245147,-1.123727,1.123727,-0.437623,0.437623
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27121,1.419447,-0.142211,-0.713079,1.477503,-0.346477,0.662612,1.231436,-0.112855,-0.608787,-0.246027,-0.409337,-0.294787,-0.245147,-1.123727,1.123727,-0.437623,0.437623
27122,-0.464399,-0.982211,-0.713079,-1.262634,-0.353532,-1.552032,-0.934416,-0.610510,-0.427893,-0.569616,-0.409337,-0.294787,-0.245147,0.889896,-0.889896,-0.437623,0.437623
27123,-0.652783,-0.142211,1.568828,-0.475788,-0.061398,0.662612,0.509486,0.934052,-0.946981,0.724743,2.949250,0.217408,0.070377,0.889896,-0.889896,-0.437623,0.437623
27124,-1.123745,0.388501,-0.713079,1.477503,-0.370207,0.662612,-1.536042,-0.112855,3.622569,-0.569616,-0.409337,-0.294787,-0.245147,0.889896,-0.889896,-0.437623,0.437623


In [20]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [21]:
model.predict_proba(X_train)

array([[9.45069312e-01, 5.49306881e-02],
       [9.69806854e-01, 3.01931455e-02],
       [9.99588476e-01, 4.11524419e-04],
       ...,
       [9.68878742e-01, 3.11212579e-02],
       [2.34807159e-01, 7.65192841e-01],
       [4.57955369e-01, 5.42044631e-01]])

In [22]:
model.predict(X_train)

array([0, 0, 0, ..., 0, 1, 1], dtype=int64)

In [23]:
accuracy_score(y_train, model.predict(X_train))

0.9035242940352429

In [24]:
X_test = pd.read_csv("test.csv")
X_test

,age,job,marital,education,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,53,blue-collar,single,secondary,105,yes,no,unknown,26,may,239,3,-1,0,unknown
1,46,blue-collar,married,secondary,450,yes,no,cellular,17,apr,198,1,-1,0,unknown
2,30,blue-collar,single,primary,5641,no,no,cellular,21,nov,695,3,-1,0,unknown
3,36,entrepreneur,married,tertiary,4576,yes,no,cellular,7,may,272,1,-1,0,unknown
4,35,blue-collar,married,secondary,412,yes,no,unknown,3,jun,316,1,-1,0,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18080,48,self-employed,married,secondary,1175,yes,yes,cellular,19,nov,62,1,-1,0,unknown
18081,42,services,married,secondary,-43,yes,yes,unknown,27,may,363,3,-1,0,unknown
18082,35,blue-collar,married,secondary,243,yes,no,cellular,20,apr,41,3,-1,0,unknown
18083,24,management,single,tertiary,66,yes,no,unknown,26,may,109,2,-1,0,unknown


In [25]:
housing_encoded = housing_encoder.transform(X_test['housing'])
for i in housing_encoded:
    X_test[i] = housing_encoded[i].astype('category')
del X_test['housing']

In [26]:
loan_encoded = loan_encoder.transform(X_test['loan'])
for i in loan_encoded:
    X_test[i] = loan_encoded[i].astype('category')
del X_test['loan']

In [27]:
X_test['marital'] = marital_encoder.transform(X_test['marital']).astype('category')
X_test.education = education_encoder.transform(X_test.education).astype('category')
X_test.job = job_encoder.transform(X_test.job).astype('category')
X_test.poutcome = poutcome_encoder.transform(X_test.poutcome).astype('category')
X_test.month = month_encoder.transform(X_test.month).astype('category')
X_test.day = day_encoder.transform(X_test.day).astype('category')
X_test.contact = contact_encoder.transform(data.contact).astype('category')

In [28]:
standarted = standart.transform(X_test)
X_test = pd.DataFrame(data=standarted)

In [30]:
X_test.columns = ['age', 'job', 'marital', 'education', 'balance', 'contact', 'day',
       'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome',
       'housing_0', 'housing_1', 'loan_0', 'loan_1']
X_test

,age,job,marital,education,balance,contact,day,month,duration,campaign,pdays,previous,poutcome,housing_0,housing_1,loan_0,loan_1
0,1.136870,-0.982211,1.568828,-0.475788,-0.400029,2.833959e-16,1.231436,-0.610510,-0.073969,0.077563,-0.409337,-0.294787,-0.245147,0.889896,-0.889896,-0.437623,0.437623
1,0.477524,-0.982211,-0.713079,-0.475788,-0.289397,2.833959e-16,0.148510,0.934052,-0.235201,-0.569616,-0.409337,-0.294787,-0.245147,0.889896,-0.889896,-0.437623,0.437623
2,-1.029552,-0.982211,1.568828,-1.262634,1.375220,2.833959e-16,0.629811,-0.021031,1.719245,0.077563,-0.409337,-0.294787,-0.245147,-1.123727,1.123727,-0.437623,0.437623
3,-0.464399,-0.673266,-0.713079,1.477503,1.033702,2.833959e-16,-1.054741,-0.610510,0.055803,-0.569616,-0.409337,-0.294787,-0.245147,0.889896,-0.889896,-0.437623,0.437623
4,-0.558591,-0.982211,-0.713079,-0.475788,-0.301583,2.833959e-16,-1.536042,-0.226236,0.228833,-0.569616,-0.409337,-0.294787,-0.245147,0.889896,-0.889896,-0.437623,0.437623
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18080,0.665909,0.115487,-0.713079,-0.475788,-0.056909,2.833959e-16,0.389160,-0.021031,-0.770019,-0.569616,-0.409337,-0.294787,-0.245147,0.889896,-0.889896,2.285073,-2.285073
18081,0.100755,-0.694305,-0.713079,-0.475788,-0.447489,2.833959e-16,1.351761,-0.610510,0.413659,0.077563,-0.409337,-0.294787,-0.245147,0.889896,-0.889896,2.285073,-2.285073
18082,-0.558591,-0.982211,-0.713079,-0.475788,-0.355776,2.833959e-16,0.509486,0.934052,-0.852601,0.077563,-0.409337,-0.294787,-0.245147,0.889896,-0.889896,-0.437623,0.437623
18083,-1.594706,0.388501,1.568828,1.477503,-0.412536,2.833959e-16,1.231436,-0.610510,-0.585192,-0.246027,-0.409337,-0.294787,-0.245147,0.889896,-0.889896,-0.437623,0.437623
